In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: GPU device not found

In [3]:
!pip install chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 32.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=2776d7d11c6664c33eb2dc300e43c96567009f9a2169a6f3f2f95a6391f3c5f5
  Stored in directory: /root/.cache/pip/wheels/fb/5d/5c/59a62d8a695285e59ec9c1f66add6f8a9ac4152499a2be0113
Successfully built chess


In [4]:
!pip install stockfish

In [5]:
import chess
import chess.engine
import chess.pgn
from multiprocessing import Pool
import numpy as np
import pandas as pd

In [6]:
pgn_file_path = "/content/drive/MyDrive/CHESS_DATASET_EVALUATE_01/lichess_elite_2020-06.pgn"

In [7]:
def extract_positions_labels_and_other_info(pgn_path, max_games=3000):
    positions = []
    labels = []
    opening = []
    whiteElo = []
    blackElo = []
    timeControl = []
    termination = []

    with open(pgn_path, "r", encoding="utf-8") as pgn:
        for game_idx in range(max_games):
            game = chess.pgn.read_game(pgn) #reads one game from the pgn at a time
            if game is None:
                break

            # Determine game outcome: 1 -> win for white, -1 -> win for black, 0 -> draw
            result = game.headers.get("Result", "*")
            if result == "1-0":
              outcome = 1
            elif result == "0-1":
              outcome = -1
            elif result == "1/2-1/2":
              outcome = 0
            else:
              continue

            # Extract positions from the game
            board = game.board()
            for move in game.mainline_moves():
                board.push(move)
                positions.append(board.fen()) # Storing the FEN representation
                labels.append(outcome) # assigning game outcome as label
                opening.append(game.headers.get("Opening", "*"))
                whiteElo.append(game.headers.get("WhiteElo", "*"))
                blackElo.append(game.headers.get("BlackElo", "*"))
                timeControl.append(game.headers.get("TimeControl", "*"))
                termination.append(game.headers.get("Termination", "*"))

    return positions, labels, opening, whiteElo, blackElo, timeControl, termination


In [8]:
positions, labels, opening, whiteElo, blackElo, timeControl, termination = extract_positions_labels_and_other_info(pgn_file_path, max_games=1400) # 82751 positions in 1000 games


len(positions), len(labels), positions[:5], labels[:5]

(117109,
 117109,
 ['rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1',
  'rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2',
  'rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBNR b KQkq - 0 2',
  'rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPP2PPP/RNBQKBNR w KQkq - 0 3',
  'rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/2N5/PPP2PPP/R1BQKBNR b KQkq - 1 3'],
 [0, 0, 0, 0, 0])

In [9]:
df = pd.DataFrame({
    'Position': positions,
    'Label': labels,
    'Opening': opening,
    'WhiteElo': whiteElo,
    'BlackElo': blackElo,
    'TimeControl': timeControl,
    'Termination': termination
})
df.shape


(117109, 7)

In [10]:
def fen_to_board(fen):
    board = []
    parts = fen.split()
    rows = parts[0].split('/')
    for row in rows:
        board_row = []
        for char in row:
            if char.isdigit():
                board_row.extend(['-'] * int(char))
            else:
                board_row.append(char)
        board.append(board_row)
    return board

In [11]:
fen_to_board("5rk1/p1p4p/1q1r1p2/6p1/8/P2PNPP1/3Q1P2/3RR1K1 b - - 0 23")

[['-', '-', '-', '-', '-', 'r', 'k', '-'],
 ['p', '-', 'p', '-', '-', '-', '-', 'p'],
 ['-', 'q', '-', 'r', '-', 'p', '-', '-'],
 ['-', '-', '-', '-', '-', '-', 'p', '-'],
 ['-', '-', '-', '-', '-', '-', '-', '-'],
 ['P', '-', '-', 'P', 'N', 'P', 'P', '-'],
 ['-', '-', '-', 'Q', '-', 'P', '-', '-'],
 ['-', '-', '-', 'R', 'R', '-', 'K', '-']]

In [12]:
def fen_to_bitboard_tensor(fen):
  board = chess.Board(fen)
  tensor = np.zeros((14, 8, 8), dtype=np.float32)

  piece_map = {
      chess.PAWN: 0, chess.KNIGHT: 1, chess.BISHOP: 2, chess.ROOK: 3, chess.QUEEN: 4, chess.KING: 5
  }

  for square in chess.SQUARES:
    piece = board.piece_at(square)
    if piece:
      row, col = divmod(square, 8)
      channel = piece_map[piece.piece_type] + (6 if piece.color == chess.BLACK else 0)
      tensor[channel, row, col] = 1

  tensor[12, :, :] = 1 if board.turn == chess.WHITE else 0

  castling = [
      board.has_kingside_castling_rights(chess.WHITE),
      board.has_queenside_castling_rights(chess.WHITE),
      board.has_kingside_castling_rights(chess.BLACK),
      board.has_queenside_castling_rights(chess.BLACK),
  ]
  tensor[13, 0, :4] = castling

  return tensor

In [13]:

for i in range(10):
    board = fen_to_board(positions[i])
  # Convert the board to a NumPy array
    board_array = np.array(board)
    print(board_array)
    print("\n")
# boardState = chess.Board(positions[0])
# boardState

[['r' 'n' 'b' 'q' 'k' 'b' 'n' 'r']
 ['p' 'p' 'p' 'p' 'p' 'p' 'p' 'p']
 ['-' '-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' 'P' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-' '-']
 ['P' 'P' 'P' 'P' '-' 'P' 'P' 'P']
 ['R' 'N' 'B' 'Q' 'K' 'B' 'N' 'R']]


[['r' 'n' 'b' 'q' 'k' 'b' 'n' 'r']
 ['p' 'p' 'p' 'p' '-' 'p' 'p' 'p']
 ['-' '-' '-' '-' 'p' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' 'P' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-' '-']
 ['P' 'P' 'P' 'P' '-' 'P' 'P' 'P']
 ['R' 'N' 'B' 'Q' 'K' 'B' 'N' 'R']]


[['r' 'n' 'b' 'q' 'k' 'b' 'n' 'r']
 ['p' 'p' 'p' 'p' '-' 'p' 'p' 'p']
 ['-' '-' '-' '-' 'p' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' 'P' 'P' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-' '-']
 ['P' 'P' 'P' '-' '-' 'P' 'P' 'P']
 ['R' 'N' 'B' 'Q' 'K' 'B' 'N' 'R']]


[['r' 'n' 'b' 'q' 'k' 'b' 'n' 'r']
 ['p' 'p' 'p' '-' '-' 'p' 'p' 'p']
 ['-' '-' '-' '-' 'p' '-' '-' '-']
 ['-' '-' '-' 'p' '-' '-' '-' '-']
 ['-' '-' '

In [14]:
!chmod +x  /content/drive/MyDrive/StockFish17/stockfish-ubuntu-x86-64-avx2

In [15]:
STOCKFISH_PATH = "/content/drive/MyDrive/StockFish17/stockfish-ubuntu-x86-64-avx2"

In [16]:
from stockfish import Stockfish

In [17]:
stockfish = Stockfish(path= STOCKFISH_PATH, depth=8, parameters={"Threads": 4, "Minimum Thinking Time": 30})

In [18]:
def get_stockfish_eval(fen, depth=5):
  with chess.engine.SimpleEngine.popen_uci(STOCKFISH_PATH) as engine:
    board = chess.Board(fen)
    info = engine.analyse(board, chess.engine.Limit(depth=depth))
    print(info)
    eval_cp = info["score"].white().score(mate_score=10000)
    return eval_cp

In [ ]:
# def get_stockfish_eval(fen, depth=10):
#   """Gets Stockfish evaluation using the stockfish library."""
#   stockfish.set_depth(depth)  # Set search depth
#   stockfish.set_fen_position(fen)  # Set the position
#   evaluation = stockfish.get_evaluation()  # Get the evaluation

#   # Extract centipawn score from evaluation
#   if evaluation['type'] == 'cp':
#     eval_cp = evaluation['value']
#   else:  # Handle mate in X moves
#     eval_cp = 10000 if evaluation['value'] > 0 else -10000

#   return eval_cp

In [19]:
eval = get_stockfish_eval("r3k3/8/8/8/8/8/7r/4K3 w - - 0 1")
print(np.tanh(eval / 400.0))

{'string': 'NNUE evaluation using nn-37f18f62d772.nnue (6MiB, (22528, 128, 15, 32, 1))', 'depth': 5, 'seldepth': 4, 'multipv': 1, 'score': PovScore(Cp(-641), WHITE), 'nodes': 14, 'nps': 7000, 'hashfull': 0, 'tbhits': 0, 'time': 0.002, 'pv': [Move.from_uci('e1f1'), Move.from_uci('h2h1'), Move.from_uci('f1g2')]}
-0.922044006208527


In [20]:
def normalize_eval(eval_cp):
    return np.tanh(eval_cp / 400.0)

In [ ]:
# def eval_score_for_all_positions(positions):
#   cp_eval_scores = [] #centipawn evaluation score
#   normalize_eval_scores = [] #normalized evaluation scores stored in a list
#   for i in range(len(positions)):
#       fen_pos = positions[i]
#       eval_cp = get_stockfish_eval(fen_pos)
#       normalized_eval = normalize_eval(eval_cp)
#       cp_eval_scores.append(eval_cp)
#       normalize_eval_scores.append(normalized_eval)
#   return cp_eval_scores, normalize_eval_scores
# def eval_score_for_all_positions(positions, depth=5, num_workers=4):
#     """Evaluate multiple positions in parallel using multiprocessing."""
#     with Pool(processes=num_workers) as pool:
#         eval_cp_scores = pool.starmap(get_stockfish_eval, [(fen, depth) for fen in positions])

#     normalize_eval_scores = [normalize_eval(cp) for cp in eval_cp_scores]

#     return eval_cp_scores, normalize_eval_scores


In [ ]:
def eval_score_for_all_positions_single(positions, depth=10):
    engine = chess.engine.SimpleEngine.popen_uci(STOCKFISH_PATH)

    eval_cp_scores = []
    normalize_eval_scores = []

    for fen in positions:
        try:
            board = chess.Board(fen)
            info = engine.analyse(board, chess.engine.Limit(depth=depth))
            eval_cp = info["score"].white().score(mate_score=10000)
            norm_score = np.tanh(eval_cp / 400.0)
            eval_cp_scores.append(eval_cp)
            normalize_eval_scores.append(norm_score)
        except Exception as e:
            print(f"⚠️ Skipping FEN due to error: {e}")
            eval_cp_scores.append(None)
            normalize_eval_scores.append(None)

    engine.quit()
    return eval_cp_scores, normalize_eval_scores


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/StockFish17/stockfish-ubuntu-x86-64-avx2 /content/

In [ ]:
sf_eval_scores, normalized_eval_scores = eval_score_for_all_positions_single(positions, 10)
df[['StockFishEval', 'NormalizedEval']] = pd.DataFrame(list(zip(sf_eval_scores, normalized_eval_scores)), columns=['StockFishEval', 'NormalizedEval'])


In [ ]:
df.to_csv('/content/drive/My Drive/Dataframe_EVALUATE_01/chess_pos_data.csv', index=False)

In [ ]:
# df['StockFishEvaluation'] = sf_eval_scores
# df['NormalizedEval'] = normalized_eval_scores
df.head(100)
# df.shape



,Position,Label,Opening,WhiteElo,BlackElo,TimeControl,Termination,StockFishEval,NormalizedEval
0,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,36,0.089758
1,rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,45,0.112028
2,rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBN...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,21,0.052452
3,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPP2PPP/RNBQKB...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,36,0.089758
4,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/2N5/PPP2PPP/R1BQ...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,33,0.082313
...,...,...,...,...,...,...,...,...,...
95,1rbq1rk1/1p1n1p1p/p2ppbp1/6B1/3NP3/6P1/PPP2PBP...,1,Sicilian Defense,2323,2415,600+0,Normal,48,0.119427
96,1rbq1rk1/1p1n1p1p/p2ppBp1/8/3NP3/6P1/PPP2PBP/R...,1,Sicilian Defense,2323,2415,600+0,Normal,7,0.017498
97,1rbq1rk1/1p3p1p/p2ppnp1/8/3NP3/6P1/PPP2PBP/R2Q...,1,Sicilian Defense,2323,2415,600+0,Normal,5,0.012499
98,1rbq1rk1/1p3p1p/p2ppnp1/8/3NP3/6P1/PPPQ1PBP/R3...,1,Sicilian Defense,2323,2415,600+0,Normal,2,0.005000


In [ ]:
fen = '8/8/8/8/8/8/1kr5/r4K1R w - - 0 1'
eval_cp = get_stockfish_eval(fen)
normalized_eval = normalize_eval(eval_cp)

print(f"Stockfish Evaluation (CP): {eval_cp}")
print(f"Normalized Evaluation: {normalized_eval}")

{'string': 'NNUE evaluation using nn-37f18f62d772.nnue (6MiB, (22528, 128, 15, 32, 1))', 'depth': 0, 'score': PovScore(Mate(-0), WHITE)}
Stockfish Evaluation (CP): -10000
Normalized Evaluation: -1.0


In [ ]:
# def count_games(pgn_path):
#     game_count = 0
#     with open(pgn_path, "r", encoding="utf-8") as pgn:
#         while True:
#             game = chess.pgn.read_game(pgn)
#             if game is None:
#                 break
#             game_count += 1
with open(pgn_file_path, "r") as pgn_file:
    game_count = sum(1 for line in pgn_file if line.startswith("[Event "))

print(f"Total number of games in the PGN file: {game_count}") # Total number of games in the PGN file: 424932

Total number of games in the PGN file: 424932


In [ ]:
# def pgn_to_dataframe(pgn_file_path):
#         pgn = open(pgn_file_path)
#         games = []
#         while True:
#             game = chess.pgn.read_game(pgn)
#             if game is None:
#                 break
#             headers = dict(game.headers)
#             moves = " ".join([move.uci() for move in game.mainline_moves()])
#             headers["moves"] = moves
#             games.append(headers)
#         return pd.DataFrame(games)

In [ ]:
# sdf = df[df['TimeControl'].str.split('+').str[0].astype(int) > 180] #games with time control > 180
# # sdf.shape
# df.shape
# sdf['TimeControl'].value_counts()
# sdf['Termination'].value_counts()

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Dataframe_EVALUATE_01/chess_pos_data.csv')
# count_games(pgn_file_path)
# df = pgn_to_dataframe(pgn_file_path)
df.head(25)
# df.shape


,Position,Label,Opening,WhiteElo,BlackElo,TimeControl,Termination,StockFishEval,NormalizedEval
0,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,36,0.089758
1,rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,45,0.112028
2,rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBN...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,21,0.052452
3,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPP2PPP/RNBQKB...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,36,0.089758
4,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/2N5/PPP2PPP/R1BQ...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,33,0.082313
5,rnbqk1nr/ppp2ppp/4p3/3p4/1b1PP3/2N5/PPP2PPP/R1...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,75,0.185333
6,rnbqk1nr/ppp2ppp/4p3/3pP3/1b1P4/2N5/PPP2PPP/R1...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,65,0.161085
7,rnbqk2r/ppp1nppp/4p3/3pP3/1b1P4/2N5/PPP2PPP/R1...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,86,0.211747
8,rnbqk2r/ppp1nppp/4p3/3pP3/1b1P4/P1N5/1PP2PPP/R...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,70,0.173235
9,rnbqk2r/ppp1nppp/4p3/3pP3/3P4/P1b5/1PP2PPP/R1B...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,97,0.237856


In [ ]:
def process_positions_with_stockfish(fen_list, depth=10):
  positions = []
  labels = []

  with chess.engine.SimpleEngine.popen_uci(STOCKFISH_PATH) as engine:
      for i, fen in enumerate(fen_list):
        # FEN TO CNN Tensor
        tensor = fen_to_bitboard_tensor(fen)

        # Get Stockfish evaluation
        info = engine.analyse(chess.Board(fen), chess.engine.Limit(depth=depth))
        eval_cp = info["score"].white().score(mate_score=10000)
        normalized_eval = normalize_eval(eval_cp)

        positions.append(tensor)
        labels.append(normalized_eval)

        if i % 100 == 0:
          print(f"Processed {i} positions...")

  return np.array(positions), np.array(labels)

In [ ]:
fen_positions = positions

# tensor_positions, labels = process_positions_with_stockfish(fen_positions, depth=10)



In [ ]:
import numpy as np
import chess

def fen_to_bitboards(fen):
    """
    Converts a FEN string into a 3D bitboard representation.
    Shape: (14, 8, 8) -> 12 for pieces, 1 for side-to-move, 1 for castling.
    """
    board = chess.Board(fen)

    # 12 planes for pieces + 1 for side-to-move + 1 for castling = 14 channels
    tensors = np.zeros((14, 8, 8), dtype=np.float32)

    # Piece-to-index mapping (White: 0-5, Black: 6-11)
    piece_map = {
        'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
        'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
    }

    # Fill the piece tensors
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            piece_idx = piece_map[piece.symbol()]
            row, col = divmod(square, 8)  # Convert 0-63 to 8x8 coordinates
            tensors[piece_idx, 7 - row, col] = 1  # Flip vertically for CNN format

    # Side to move (1 if White, 0 if Black)
    tensors[12, :, :] = 1 if board.turn == chess.WHITE else 0

    # Castling rights (Four separate planes)
    castling_rights = [chess.BB_H1, chess.BB_A1, chess.BB_H8, chess.BB_A8]  # KQkq
    for i, right in enumerate(castling_rights):
        tensors[13, :, :] = 1 if board.castling_rights & right else 0

    return tensors

# Example FEN
"""
fen = "rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR w - - 0 1"
bitboard_tensor = fen_to_bitboards(fen)

print("Bitboard Shape:", bitboard_tensor.shape)  # (14, 8, 8)
print(bitboard_tensor[0])  # White Pawn Plane
print("\n")
print(bitboard_tensor[6])  # Black Pawn Plane
print("\n")
print(bitboard_tensor[13])
"""

'\nfen = "rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR w - - 0 1"\nbitboard_tensor = fen_to_bitboards(fen)\n\nprint("Bitboard Shape:", bitboard_tensor.shape)  # (14, 8, 8)\nprint(bitboard_tensor[0])  # White Pawn Plane\nprint("\n")\nprint(bitboard_tensor[6])  # Black Pawn Plane\nprint("\n")\nprint(bitboard_tensor[13])\n'

In [ ]:
bitboard_list = []

for fen in positions:
    bitboard = fen_to_bitboards(fen)
    bitboard_list.append(bitboard)

bitboard_array = np.array(bitboard_list)

np.save('/content/drive/MyDrive/Dataframe_chess_eval/bitbrd.npy', bitboard_array)

print(bitboard_array.shape)

(117109, 14, 8, 8)


In [ ]:
df.head(20)

,Position,Label,Opening,WhiteElo,BlackElo,TimeControl,Termination,StockFishEval,NormalizedEval
0,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,36,0.089758
1,rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,45,0.112028
2,rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBN...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,21,0.052452
3,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPP2PPP/RNBQKB...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,36,0.089758
4,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/2N5/PPP2PPP/R1BQ...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,33,0.082313
5,rnbqk1nr/ppp2ppp/4p3/3p4/1b1PP3/2N5/PPP2PPP/R1...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,75,0.185333
6,rnbqk1nr/ppp2ppp/4p3/3pP3/1b1P4/2N5/PPP2PPP/R1...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,65,0.161085
7,rnbqk2r/ppp1nppp/4p3/3pP3/1b1P4/2N5/PPP2PPP/R1...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,86,0.211747
8,rnbqk2r/ppp1nppp/4p3/3pP3/1b1P4/P1N5/1PP2PPP/R...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,70,0.173235
9,rnbqk2r/ppp1nppp/4p3/3pP3/3P4/P1b5/1PP2PPP/R1B...,0,"French Defense: Winawer Variation, Poisoned Pa...",2413,2254,180+2,Normal,97,0.237856


In [ ]:
print(bitboard_array[2])

[[[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 1. 1. 0. 0. 1. 1. 1.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 1. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 1. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 1.]]

 [[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 

In [ ]:
newDF = df.copy()
# newDF['Bitboard'] = bitboard_array
newDF['Bitboard'] = [x.decode('utf-8') if isinstance(x, bytes) else x for x in bitboard_array]
# newDF['Bitboard'].head()
newDF.tail()

,Position,Label,Opening,WhiteElo,BlackElo,TimeControl,Termination,StockFishEval,NormalizedEval,Bitboard
117104,8/1p6/8/k1K5/8/8/8/8 w - - 4 63,-1,Sicilian Defense: Najdorf Variation,2526,2560,180+0,Time forfeit,-581,-0.896186,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0..."
117105,8/1p6/8/k7/2K5/8/8/8 b - - 5 63,-1,Sicilian Defense: Najdorf Variation,2526,2560,180+0,Time forfeit,-556,-0.883171,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0..."
117106,8/1p6/8/8/k1K5/8/8/8 w - - 6 64,-1,Sicilian Defense: Najdorf Variation,2526,2560,180+0,Time forfeit,-650,-0.925346,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0..."
117107,8/1p6/8/2K5/k7/8/8/8 b - - 7 64,-1,Sicilian Defense: Najdorf Variation,2526,2560,180+0,Time forfeit,-655,-0.927122,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0..."
117108,8/8/8/1pK5/k7/8/8/8 w - - 0 65,-1,Sicilian Defense: Najdorf Variation,2526,2560,180+0,Time forfeit,-648,-0.924624,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0..."


In [ ]:
!pip install tensorflow

In [ ]:
!pip install keras

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Dataframe_EVALUATE_01/chess_pos_data.csv')
df.head(10)
newDF = df.copy()
# newDF.to_csv('/content/drive/My Drive/Dataframe_EVALUATE_01/final_processed_data.csv', index=False)

In [ ]:
bitboard_array = np.load('/content/drive/MyDrive/Dataframe_chess_eval/bitbrd.npy')


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam # type: ignore

X = newDF[['NormalizedEval']].to_numpy()
# Y = newDF[['Bitboard']].to_numpy()
Y = bitboard_array
Y = Y.transpose(0,2,3,1)

# Split into training and temp (for validation and test)
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.2, random_state=42)

# Further split temp into validation and test (50% each of the temp data, so 0.1 each of the total data)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)

# print(X)
# print(Y)
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(8, 8, 14), padding="same"),
    Conv2D(64, (3, 3), activation='relu', padding="same"),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='tanh')  # Output layer (Normalized evaluation score)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 8, 8, 32)       │         4,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 8, 8, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       524,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 555,297 (2.12 MB)

 Trainable params: 555,297 (2.12 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_mae', patience=3, restore_best_weights=True)

In [ ]:
history = model.fit(Y_train, X_train, epochs=15, batch_size=32, validation_data=(Y_test, X_test), callbacks=[early_stop])

Epoch 1/15
2928/2928 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - loss: 0.1648 - mae: 0.2919 - val_loss: 0.0870 - val_mae: 0.2086
Epoch 2/15
2928/2928 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.0692 - mae: 0.1856 - val_loss: 0.0572 - val_mae: 0.1659
Epoch 3/15
2928/2928 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.0429 - mae: 0.1458 - val_loss: 0.0484 - val_mae: 0.1523
Epoch 4/15
2928/2928 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 0.0323 - mae: 0.1258 - val_loss: 0.0424 - val_mae: 0.1398
Epoch 5/15
2928/2928 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.0259 - mae: 0.1135 - val_loss: 0.0371 - val_mae: 0.1309
Epoch 6/15
2928/2928 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 0.0222 - mae: 0.1048 - val_loss: 0.0349 - val_mae: 0.1251
Epoch 7/15
2928/2928 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.0185 - mae: 0.0958 - val_loss: 0.0354 - val_mae: 0.1259
Epoch 8/15
2928/2928 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.0167 - mae: 0.0915 - val_loss: 0.0330 - val_mae: 0.1198
Epoch 9/15
2928/2928 ━━━━━━━━━━━━━

In [ ]:
model.save('/content/drive/MyDrive/Dataframe_chess_eval/model.keras')

In [ ]:
import tensorflow.keras as keras
model = keras.models.load_model('/content/drive/MyDrive/Dataframe_chess_eval/model.keras')

In [ ]:
loss, mae = model.evaluate(Y_test, X_test)
print(f"Test MAE: {mae}")

732/732 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 0.0276 - mae: 0.1061
Test MAE: 0.10579194128513336


In [ ]:
fen = "5rk1/p1p4p/1q1r1p2/6p1/8/P2PNPP1/3Q1P2/3RR1K1 b - - 0 23"
# fen = "5rk1/p1p4p/3r1p2/q5p1/8/P2PNPP1/5P2/3RR1K1 w - - 0 25"
# fen = "4k3/7R/8/8/8/8/8/R3K3 w Q - 0 1"
# fen = "rnbr2k1/ppp1pppp/1q6/8/8/1Q6/PPP1PPPP/RNBR2K1 b - - 0 1"

tensors_input = fen_to_bitboards(fen)
tensors_input = np.expand_dims(tensors_input, axis=0)
# Reshape tensors_input to match the expected input shape of the model
tensors_input = tensors_input.transpose(0, 2, 3, 1)
predicted_eval = model.predict(tensors_input)
print(predicted_eval)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
[[0.18054073]]


In [ ]:
import matplotlib.pyplot as plt


In [ ]:
# Assuming 'history' is the variable storing the training history
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Test MAE')  # Add this line

# plt.xlabel('Epoch')
# plt.ylabel('MAE')
# plt.title('Training and Test MAE')
# plt.legend()
# plt.show()

NameError: name 'history' is not defined

In [ ]:
df.to_csv('/content/drive/MyDrive/Dataframe_chess_eval/datafrm.csv', index=False)